In [1]:
import math
import pandas as pd
import numpy as np
from datetime import datetime as dt
import os
import matplotlib.pyplot as plt
import pandas_datareader.data as web
plt.style.use('fivethirtyeight')

/opt/anaconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
stockStartDate = '2013-01-01'
today = dt.today().strftime('%Y-%m-%d')
df = web.DataReader("TSLA", data_source='yahoo', start=stockStartDate , end=today)

In [3]:
df.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-07-06,1377.790039,1266.040039,1276.689941,1371.579956,20569900,1371.579956
2020-07-07,1429.500000,1336.709961,1405.010010,1389.859985,21489700,1389.859985
2020-07-08,1417.260010,1311.339966,1405.000000,1365.880005,16311300,1365.880005
2020-07-09,1408.560059,1351.280029,1396.989990,1394.280029,11717600,1394.280029
2020-07-10,1548.920044,1376.010010,1396.000000,1544.650024,23281000,1544.650024


In [4]:
df_ohlc = df['Adj Close'].resample('10D').ohlc()
df_volume = df['Volume'].resample('10D').sum()
print(df_ohlc.head())
print(df_volume.head())

                 open       high        low      close
Date                                                  
2012-12-31  33.869999  35.360001  33.639999  33.639999
2013-01-10  33.529999  34.520000  32.910000  34.520000
2013-01-20  35.189999  38.029999  35.189999  37.950001
2013-01-30  37.520000  39.480000  37.509998  39.240002
2013-02-09  38.419998  38.450001  37.040001  37.040001
Date
2012-12-31     5629700
2013-01-10    11405000
2013-01-20    10155300
2013-01-30     9637900
2013-02-09     9502700
Freq: 10D, Name: Volume, dtype: int64


In [5]:
import bs4 as bs
import pickle
import requests

In [6]:
def save_sp500_ticker():
    resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class':'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
        
    tickers = [s.strip() for s in tickers]

    with open("sp500tickers.pickle", 'wb') as f:
        pickle.dump(tickers, f)
        
    return tickers
        

In [7]:
save_sp500_ticker()

['MMM',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AMD',
 'AAP',
 'AES',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALXN',
 'ALGN',
 'ALLE',
 'ADS',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'AOS',
 'APA',
 'AIV',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BLL',
 'BAC',
 'BK',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'BIIB',
 'BLK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BF.B',
 'CHRW',
 'COG',
 'CDNS',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CTL',
 'CERN',
 'CF',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CMA',
 'CAG',
 'CXO'

In [13]:
def get_data_from_yahoo(reload_sp500 = False):
    tickers = save_sp500_ticker()
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')
        
    start = '2013-01-01'
    end = dt.today().strftime('%Y-%m-%d')
    
    for ticker in tickers[:20]:
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df = web.DataReader(ticker, 'yahoo', start = start, end = end)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))
            
get_data_from_yahoo()        

In [14]:
def compile_data():
    tickers = save_sp500_ticker()
    main_df = pd.DataFrame()
    for count,ticker in enumerate(tickers[:20]):
        df = pd.read_csv('stock_dfs/{}.csv'.format(ticker))
        df.set_index('Date', inplace = True)
        df.rename(columns = {'Adj Close' : ticker}, inplace = True)
        df.drop(['Open','High','Low','Close','Volume'], 1, inplace = True)
        
        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df, how = 'outer')
            
        if count%10 == 0:
            print(count)
    print(main_df.head())
    main_df.to_csv('sp500all.csv')

In [15]:
compile_data()

0
10
                  MMM        ABT       ABBV   ABMD        ACN       ATVI  \
Date                                                                       
2012-12-31  76.014526  27.044691        NaN  13.44  56.649178   9.934611   
2013-01-02  77.594589  27.580906  25.678120  13.69  58.829960  10.243313   
2013-01-03  77.504532  28.630791  25.466085  13.45  58.617001  10.299441   
2013-01-04  78.077629  28.458672  25.144379  13.41  58.940701  10.449116   
2013-01-07  78.175842  28.691023  25.195559  13.25  58.685146  10.458469   

                 ADBE   AMD        AAP       AES        AFL          A  \
Date                                                                     
2012-12-31  37.680000  2.40  70.972847  8.384384  19.854929  27.048517   
2013-01-02  38.340000  2.53  71.110153  8.619459  20.210012  27.669559   
2013-01-03  37.750000  2.49  71.110153  8.572444  19.697935  27.768665   
2013-01-04  38.130001  2.59  72.218658  8.776177  19.451244  28.317034   
2013-01-07  37.939